In [71]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import psycopg2
from sklearn import linear_model 
from sklearn.decomposition import PCA
import matplotlib. pyplot as plt 
import statsmodels.api as sm
from config import password

In [37]:
# Load the pga data set dataset from database 
connection_string = f"host=red-team-final.c3yshvuxvm9x.us-east-2.rds.amazonaws.com dbname=red-team port=5432 user=postgres password={password}"
connection = psycopg2.connect(connection_string)

In [38]:
# Establish connection and db
sql = "select * from pga_stats"
pga_df = pd.read_sql(sql, connection)

In [39]:
# Show df
pga_df

,SG_off_tee_AVERAGE,SG_tee_green_AVERAGE,driving_dist_AVG.,pct_ydg_tee_AVG (%),driving_320+_%,SG_approach_green_AVERAGE,GIR_pct_%,GOFIR_pct_%,GIR_pct_fwy_%,GIR_pct_not_fwy_%,...,pct_sand_save_%,scrambling_%,SG_putt_AVERAGE,pct_one_putt_%,pct_three_putt_avoid_%,putt_avg_AVG,putts_per_rd_AVG,bogey_avoid_% MAKES BOGEY,btb_ratio_BIRDIE TO BOGEY RATIO,money_MONEY
0,1.144,2.099,321.1,66.80,51.47,0.696,66.67,80.34,80.87,50.25,...,48.89,61.27,0.435,41.01,3.27,1.752,28.74,14.54,1.84,5443091
1,0.841,1.700,307.1,64.01,29.17,0.582,72.11,85.90,83.80,57.32,...,50.00,63.07,-0.019,38.43,2.55,1.750,29.04,14.00,1.79,3556232
2,0.774,1.064,318.2,64.96,48.68,0.178,65.35,74.37,79.47,53.10,...,57.35,57.81,0.040,42.11,3.36,1.743,28.76,17.11,1.48,1782342
3,0.610,0.833,299.9,63.64,13.28,0.553,73.18,79.81,80.52,65.29,...,46.05,61.49,-0.013,37.67,2.52,1.756,29.30,12.50,1.91,1482028
4,0.567,0.563,287.4,60.36,8.33,-0.088,64.81,68.25,72.54,51.09,...,52.50,60.53,-0.793,39.12,2.78,1.779,29.25,15.74,1.15,164271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,-0.063,0.284,300.9,63.32,16.67,0.585,66.43,73.44,76.26,54.23,...,56.10,58.99,-0.113,36.55,3.22,1.807,29.45,16.26,1.17,824162
461,-0.417,-0.168,278.7,60.89,1.92,-0.006,63.53,68.20,73.63,48.14,...,54.17,62.30,0.451,42.02,1.42,1.748,28.32,15.03,1.27,942372
462,0.022,-0.402,285.1,62.05,1.69,-0.429,64.69,69.61,77.00,52.37,...,51.35,58.67,-0.006,38.51,2.73,1.779,29.15,16.95,1.14,332352
463,0.255,0.325,296.4,63.35,13.27,0.251,66.07,71.07,76.71,51.13,...,51.45,56.98,0.154,38.78,3.05,1.772,29.08,17.05,1.24,1943309


In [40]:
# Define the features set as everything but money 
X = pga_df.copy()
X = X.drop("money_MONEY", axis=1)
X.head()

,SG_off_tee_AVERAGE,SG_tee_green_AVERAGE,driving_dist_AVG.,pct_ydg_tee_AVG (%),driving_320+_%,SG_approach_green_AVERAGE,GIR_pct_%,GOFIR_pct_%,GIR_pct_fwy_%,GIR_pct_not_fwy_%,...,SG_ATG_AVERAGE,pct_sand_save_%,scrambling_%,SG_putt_AVERAGE,pct_one_putt_%,pct_three_putt_avoid_%,putt_avg_AVG,putts_per_rd_AVG,bogey_avoid_% MAKES BOGEY,btb_ratio_BIRDIE TO BOGEY RATIO
0,1.144,2.099,321.1,66.80,51.47,0.696,66.67,80.34,80.87,50.25,...,0.259,48.89,61.27,0.435,41.01,3.27,1.752,28.74,14.54,1.84
1,0.841,1.700,307.1,64.01,29.17,0.582,72.11,85.90,83.80,57.32,...,0.326,50.00,63.07,-0.019,38.43,2.55,1.750,29.04,14.00,1.79
2,0.774,1.064,318.2,64.96,48.68,0.178,65.35,74.37,79.47,53.10,...,0.112,57.35,57.81,0.040,42.11,3.36,1.743,28.76,17.11,1.48
3,0.610,0.833,299.9,63.64,13.28,0.553,73.18,79.81,80.52,65.29,...,-0.329,46.05,61.49,-0.013,37.67,2.52,1.756,29.30,12.50,1.91
4,0.567,0.563,287.4,60.36,8.33,-0.088,64.81,68.25,72.54,51.09,...,0.084,52.50,60.53,-0.793,39.12,2.78,1.779,29.25,15.74,1.15


In [41]:
# Define the target set as Money/ Earning PGA
y = pga_df["money_MONEY"].ravel()
y[:5]

array([5443091, 3556232, 1782342, 1482028,  164271], dtype=int64)

In [50]:
#Split Data into testing and training 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [67]:
#call Linear Regression model
regr = linear_model.LinearRegression()

## train data to model 
regr.fit(X_train, y_train)


LinearRegression()

In [68]:
print('Intercept: \n', model.intercept_)
print('Coefficients: \n', model.coef_)

Intercept: 
 37146355.71658906
Coefficients: 
 [ 8.91380409e+05  4.68946741e+05 -3.55960401e+04  1.91278066e+05
  3.26475703e+04  1.24350819e+06 -1.69302890e+05 -8.68166541e+04
 -2.01981039e+04 -4.88221387e+03 -4.96867297e+03  3.05117438e+02
  1.18158587e+06 -1.39809324e+03  2.84982524e+03  1.55741261e+06
 -2.62667266e+05  2.75792344e+05 -8.96096135e+06  7.94475110e+03
  1.80099853e+05  4.05460252e+06]


In [73]:
model = sm.OLS(y_train, X_train).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.846
Model:                            OLS   Adj. R-squared (uncentered):              0.835
Method:                 Least Squares   F-statistic:                              81.13
Date:                Thu, 06 May 2021   Prob (F-statistic):                   5.58e-118
Time:                        09:58:21   Log-Likelihood:                         -5317.3
No. Observations:                 348   AIC:                                  1.068e+04
Df Residuals:                     326   BIC:                                  1.076e+04
Df Model:                          22                                                  
Covariance Type:            nonrobust                                                  
                                      coef    std err          t      P>|t|      [0.025      0.975]
--------------------